<a href="https://colab.research.google.com/github/ivalexander13/mrsa-nisin-resistance/blob/master/MRSA_Nisin_Resistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [0]:
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.stats as stats
plt.style.use('fivethirtyeight')
import pandas as pd
import seaborn as sns
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [0]:
raw1 = pd.read_csv("https://raw.githubusercontent.com/ivalexander13/mrsa-nisin-resistance/master/samples_raw/GSM3147945_WT1_S7.txt", delimiter = "\t")

In [6]:
raw1

,Feature ID,Wild-type - WT1_S7 (GE) - Expression values,Wild-type - WT1_S7 (GE) - Unique gene reads,Wild-type - WT1_S7 (GE) - Total gene reads
0,SAOUHSC_01037,10,10,10
1,SAOUHSC_00303,4,4,4
2,SAOUHSC_00978,94,94,94
3,SAOUHSC_02313,3,3,3
4,SAOUHSC_02518,6,6,6
...,...,...,...,...
2867,SAOUHSC_02939,0,0,0
2868,SAOUHSC_02948,0,0,0
2869,SAOUHSC_02950,1,1,1
2870,SAOUHSC_A01436,0,0,0
